## Setup environment and Project Client

In [ ]:
# Install dependencies

%pip install azure-identity
%pip install azure-ai-projects
%pip install python-dotenv

In [ ]:
# Load environment 

from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
# Create the project client

import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
project_client = AIProjectClient.from_connection_string(
    credential=credential, conn_str=os.environ["AZURE_AI_PROJECT_CONNECTION_STRING"]
)

## Prep the File and Vector Store (agent-level vector store)

In [ ]:
from azure.ai.projects.models import FilePurpose

# Upload file

file = project_client.agents.upload_file_and_poll(
    file_path="./files/food-holidays.txt",
    purpose=FilePurpose.AGENTS
)
print(f"Uploaded file, file ID: {file.id}")

# Create a vector store and attach file

vector_store = project_client.agents.create_vector_store_and_poll(
    file_ids=[file.id],
    name="holidays-vectorstore"
)
print(f"Created vector store, vector store ID: {vector_store.id}")

## Create the Agent, Thread, and add a User Message

In [ ]:
from azure.ai.projects.models import FileSearchTool
from datetime import date

# Initialize the Agent's File Search Tool with access to the vector store

file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

# Create the Agent

agent = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="holidays-recipe-agent",
    instructions=f"You are a helpful assistant. You answer questions about upcoming events based on your calendar data. The current date is {date.today()}. Do not make up events.",
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources
)
print(f"Created agent, agent ID: {agent.id}")

In [ ]:
from azure.ai.projects.models import MessageTextContent

# Create a thread

thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

# Add a user message to the thread

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="What food holidays are coming up over the next week?"
)

## Run the Thread and View Results

In [ ]:
# Create a run and wait for it to finish

run = project_client.agents.create_and_process_run(
    thread_id=thread.id,
    agent_id=agent.id
)
print(f"Completed run, run ID: {run.id}")

In [ ]:
# Print run results and citations

messages = project_client.agents.list_messages(thread_id=thread.id)
for m in reversed(messages.data):
    last_message_content = m.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"\n{m.role.upper()}: {last_message_content.text.value}")

print("\nCitations:")
for c in messages.file_citation_annotations:
    print(f"- Citation snippet: '{c.text}' from file ID: {c.file_citation['file_id']}")

## Add File via Message Attachment (thread-level managed vector store)

In [ ]:
from azure.ai.projects.models import MessageAttachment

# Upload a new file for use as a message attachment

message_file = project_client.agents.upload_file_and_poll(
    file_path="./files/birthdays.txt",
    purpose=FilePurpose.AGENTS
)
print(f"Uploaded file, file ID: {message_file.id}")

# Add a user message to the tread with the file attachment

attachment = MessageAttachment(
    file_id=message_file.id,
    tools=FileSearchTool().definitions
)
message_attachment = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="I need to prepare something special for a teammate's birthday. What food holidays are on the next two team members' birthdays?",
    attachments=[attachment]
)

## Run the Thread and View Results

In [ ]:
# Create a run and wait for it to finish

run = project_client.agents.create_and_process_run(
    thread_id=thread.id,
    agent_id=agent.id
)
print(f"Completed run, run ID: {run.id}")

In [ ]:
# Print run results and citations

messages = project_client.agents.list_messages(thread_id=thread.id)
for m in reversed(messages.data):
    last_message_content = m.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"\n{m.role.upper()}: {last_message_content.text.value}")

print("\nCitations:")
for c in messages.file_citation_annotations:
    print(f"- Citation snippet: '{c.text}' from file ID: {c.file_citation['file_id']}")

## Cleanup

In [ ]:
# Explicit cleanup

project_client.agents.delete_vector_store(vector_store_id=vector_store.id)
project_client.agents.delete_file(file_id=file.id)
project_client.agents.delete_file(file_id=message_file.id)
project_client.agents.delete_thread(thread_id=thread.id)
project_client.agents.delete_agent(agent_id=agent.id)

# Clean up EVERYTHING...

# # Cleanup any vector stores
# vector_stores = project_client.agents.list_vector_stores()
# for vs in vector_stores.data:
#     print(f"Deleting vector store: {vs.id}")
#     project_client.agents.delete_vector_store(vector_store_id=vs.id)
# # Cleanup any files
# files = project_client.agents.list_files(purpose=FilePurpose.AGENTS)
# for f in files.data:
#     print(f"Deleting file: {f.id}")
#     project_client.agents.delete_file(file_id=f.id)
# # Cleanup any agents
# agents = project_client.agents.list_agents()
# for a in agents.data:
#     print(f"Deleting agent: {a.id}")
#     project_client.agents.delete_agent(agent_id=a.id)